# Gradient Descent

We now know how to *make* a neural network, and how to have that neural network *predict* an outcome from input data.  However, unless we're very lucky, that prediction is unlikely to be worth much of anything.

Therefore, we need a way of *training* the neural network -- that is, feeding it some questions and correct answers, having it change to accommodate that data, and then (hopefully) making good predictions about future data afterwards.

## Cost Functions

The idea is simple enough, if you understand multivariable calculus.  If you don't, this part might not be that fun.

What we will do is define a *cost function* -- that is, a function $C(\Theta,X,Y)$, where $\Theta$ represents the weights and biases of the neural network, $X$ is the input data, and $Y$ is the correct answer.  The *cost* $C(\Theta,X,Y)$ will somehow measure the error of the prediction that $\Theta$ makes from $X$, as compared to the correct answer $Y$.

So for example, if our network predicts the value 1, but the correct answer was 0, that cost would be higher than if we predicted 0.5 or 0.25.  If we predict the answer 0, the cost should be zero.  It is vital that at all times, the cost is *positive*, unless the answer is exactly right, in which case the cost is *zero*.

Of course typically $X$ and $Y$ will consist of several examples, so the total cost $C$ will somehow measure all of the errors at once, typically just by adding them all up.  We will talk about different cost functions in the next notebook, but for now just assume we have one in mind.

## Gradients

The point of the cost function is to estimate how wrong we are.  The point of training is to make this cost go *down*.  Assuming the cost function is well-designed, this means our network is improving at its task, which is to predict $Y$ from $X$.  Thus, we want to move $\Theta$ (the parameters for the neural network) so that the cost function is as low as possible.

In high school algebra or calculus, you may have learned how to minimize certain kinds of functions.  Unfortunately in virtually all cases those techniques are not going to be useful -- $\Theta$ has so many variables, and the cost function (in terms of $\Theta$) is so complicated, that symbolic methods don't work.

Thus we use what is called **gradient descent**.

The gradient of a function is a vector which points in the *direction of steepest ascent*.  What this means is that if you want to make the function go up, move $\Theta$ forward along the gradient, and if you want to make the function go down, move $\Theta$ backward along the gradient.

So the gradient descent algorithm works like this:
1. Compute the gradient of the cost function, with respect to $\Theta$. Call it $G$.
2. Replace $\Theta$ with $\Theta-\alpha\cdot G$, where $\alpha$ is some small positive number.
3. Go to 1

When to stop the gradient descent algorithm is a bit tricky, but basically you go until you've basically stopped moving.  If the gradient was ever zero, you would have converged at a (local) minimum (hopefully) but in practice this doesn't happen.  So we just keep going and going until the computer is tired.

## Learning Rates

There is a first question you might ask.  Why use $\alpha$?  And why should it be small?

Well, the idea is that the gradient changes as you move, so if you shoot out too far along the first $G$ you see, the first bit of motion might be useful, but as the gradient changes, you might actually be moving in the wrong direction, and end up making the cost function go up!

However, the bigger your $\alpha$, the faster you train, so this is our first **hyper parameter**.  You want your $\alpha$ to be as big as possible, unless it causes your function to blow up.  We'll talk about how to choose $\alpha$ later, but for now, just playing with it is good enough.

## Back Propagation

The more pressing question is how to compute the gradient at all.  This is somewhat involved, and uses the chain rule.  If you don't remember your multivariable calculus, or were never that good at it, this part is particularly unpleasant.  In fact, if you don't care, just skip this section and look at the code at the end, it won't hurt you (although at some point you'll want to understand how this works, at least roughly).

The basic idea is this.

Let $w_l$ and $b_l$ be the weights and biases at the final level.  The prediction function is exactly $y_l=f_l(w_ly_{l-1}+b_l)$, where $y_{l-1}$ is the output of the $l-1$*th* level, and does not depend on $w_l$ or $b_l$ in any way, and $f_l$ is the activation function at the final level.

The cost of the prediction $y_l$ is given by some function $C(z)$.  So: $$\dfrac{\partial C}{\partial w_l}=C'(y_l)\cdot \dfrac{\partial y_l}{\partial w_l} = C'(y_l) \cdot y_{l-1} \cdot f_l'(w_l y_{l-1}+b_l)= C'(y_l) \cdot y_{l-1} \cdot f_l'(z_l)$$
$$\dfrac{\partial C}{\partial b_l}=C'(y_l)\cdot \dfrac{\partial y_l}{\partial b_l} = C'(y_l) \cdot f_l'(w_ly_{l-1}+b_l) = C'(y_l) \cdot f_l'(z_l)$$

Here and throughout, $z_i=y_{i-1}w_i+b_i$ -- the thing you plug into $f_i$ to get $y_i$.

The $y_{l-1}$ is present in the top line because it is multiplied by $w_l$, but because $b_l$ is alone in the expression for $y_l$, there is no chain rule problem there.

Computing the gradient for the final level, as above, is not too bad.  The earlier levels are more involved.  The key identity is the chain rule, written nicely in Leibniz notation as follows:

$$\dfrac{\partial C}{\partial w_i}=\dfrac{\partial C}{\partial y_l}\cdot \dfrac{\partial y_l}{\partial w_i}=\dfrac{\partial C}{\partial y_l}\cdot \dfrac{\partial y_l}{\partial y_{l-1}}\cdots\dfrac{\partial y_i}{\partial w_i}$$

$$\dfrac{\partial C}{\partial b_i}=\dfrac{\partial C}{\partial y_l}\cdot \dfrac{\partial y_l}{\partial b_i}=\dfrac{\partial C}{\partial y_l}\cdot \dfrac{\partial y_l}{\partial y_{l-1}}\cdots\dfrac{\partial y_i}{\partial b_i}$$

So we only need to compute all these derivative pieces, which turn out to be fairly easy.

$\frac{\partial C}{\partial y_l}=C'(y_l)$, and this depends on the cost function, but will always be easy to compute for us.

$\frac{\partial y_i}{\partial w_i}=y_{i-1}f'(z_i)$, where $y_{-1}$ is the input to the network, and $\frac{\partial y_i}{\partial b_i}=f'(z_i)$.  The presence of $y_{i-1}$ (or not) is because of how $z_i$ is defined, as before.

Finally, $y_j=f(w_jy_{j-1}+b_j)$, so $\frac{\partial y_j}{\partial y_{j-1}}=w_jf'(z_j)$.

This is all fine, and is approximately true.  It is unfortunately true that all of these variables -- $y$, $z$, $w$, and $b$ -- are vectors, not numbers, so we have to be fairly careful about things like the order of multiplication.  However, once everything is laid out in the correct order, the formulas are correct as stated.

## The Code

Now we can actually write our code.  We assume our cost function looks like this (we'll define some actual ones in the next notebook):

In [1]:
def cost(predicted, actual, diff=False):
    if diff:
        # somehow compute the derivative of the cost ...
        return deriv_of_cost
    else:
        # somehow compute the actual cost
        return actual_cost

In what follows, `bp_grad` will be the successive matrices $\frac{\partial C}{\partial y_i}$, where $i$ goes from $0$ to $l$, or in code, `0` to `L-1`.

In [2]:
def back_prop(weights, biases, acts, cost_function,
        train_X, train_Y,
        x, y, z):
    L = len(weights) # number of layers
    
    cost_diff = cost_function(y[-1], train_Y, diff=True)
    
    # Gradient of cost at each level
    bp_grad = [0] * L
    
    # The last level is special
    bp_grad[L-1] = cost_diff * acts[L-1](z[L-1], y[L-1], diff=True)
    
    # The rest of the levels are just gotten by propagating backward
    for i in range(L-2, -1, -1):
        scaled_grad = bp_grad[i+1] * acts[i+1](z[i+1], y[i+1], diff=True)
        bp_grad[i] = np.dot(scaled_grad, weights[i+1].T)
    
    # Now adjust for the weights and biases themselves
    bp_grad_w = [0] * L
    bp_grad_b = [0] * L
    
    for i in range(0, L):
        scaled_grad = bp_grad[i] * acts[i](z[i], y[i], diff=True)

        bp_grad_w[i] = np.dot(x[i].T, scaled_grad)
        
        relevant_ones = np.ones((1, x[i].shape[0]))
        bp_grad_b[i] = np.dot(relevant_ones, scaled_grad)
        
    return bp_grad_w, bp_grad_b